In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col("datum").dt.year() >= 1918
).drop_nulls(
    subset=["datum"]
).sort(
    by="datum"
)

In [4]:
import re

In [5]:
def vroce(retezec):
    roky = []
    fraze = [
        r'[Rr]ok \d{2,4}(?!\sKč|\skorun)',
        r'[Vv] roce \d{2,4}(?!\sKč|\skorun)',
        r'[Rr]oku \d{2,4}(?!\sKč|\skorun)',
        r'[Rr]okem \d{2,4}(?!\sKč|\skorun)'
    ]
    for f in fraze:
        nalezy = re.findall(f,retezec)
        pridej = [int(re.search(r'\d{2,4}', x).group()) for x in nalezy]
        pridej = [int(f'19{x}') if 18 <= x < 100 else int(f'20{x}') if x < 17 else x for x in pridej]
        roky += pridej

    roky = [x for x in roky if 1300 < x < 2500]
    roky.sort()
    return roky

In [6]:
test = 'V roce 2011 bylo dobře, zato v roce 2012 bylo ještě líp, ovšem roku 2013 se to začalo kazit a rokem 2014 dobré časy skončily. Rok 39 byl těžký, rok 89 veselý. Rok 14 byl rokem začátku války. Vydělávám za rok 2029 Kč, přesněji vydělávám za rok 2030 korun. Píše se rok 2032, nejvíc od roku 2031.'

In [7]:
vroce(test)

[1939, 1989, 2011, 2012, 2013, 2014, 2014, 2031, 2032]

In [8]:
test2 = 'Žádná zmínka'

In [9]:
vroce(test2)

[]

In [10]:
df = df.with_columns(pl.col("text").map_elements(vroce, return_dtype=pl.List(pl.Int32)).alias("roky"))

In [68]:
vyvoj = df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    (pl.col("roky") - pl.col("datum").dt.year()).alias("rozdil")
).filter(
    pl.col("rozdil").is_between(-100,100)
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('rozdil').mean()
)

alt.Chart(vyvoj.to_pandas(),width=800,title="Během revolucí se nemluví o budoucnosti, ale o minulosti").mark_bar().encode(alt.X('datum:T'),alt.Y('rozdil:Q'))

alt.Chart(...)

In [13]:
vyvoj2 = df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    pl.col("roky") - pl.col("datum").dt.year()
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('roky').quantile(0.95)
)

In [14]:
alt.Chart(vyvoj2.to_pandas(),width=800).mark_bar().encode(alt.X('datum:T'),alt.Y('roky:Q'))

alt.Chart(...)

In [15]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 2000).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [82]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 1918).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [76]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 1939).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [77]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 1945).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [70]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 1945).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [80]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 1948).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [72]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 1968).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [198]:
def hledej(text, n=3):
    nalezy = df.sort(by="datum").filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-200:] if len(before) > 200 else before
        after_snippet = after[:200] if len(after) > 200 else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [200]:
hledej(r'[Rr]o[ck]\w{0,4} 2000', n=10)

Kdo: Posl. dr. Radda (německy)
Kde: 1920ns_ps_stenprot_127schuz_s127012.htm
Kdy: 1922-01-26
Co: u poslanců, aby se později podle finančního plánu Rašínova provedlo všude zkrácení mezd dělnických; a myslím, že již to možno označiti za demagogii. Velectění! Je směšno, předpisujete-li poslancům za rok 2000 korun za legitimaci k jízdě, poněvadž to při výšce diet přece zcela samozřejmě nemůže ani přijíti v úvahu jako nějaká srážka. Jsem přesvědčen, že nebude poslance v tomto parlamentu, který by tuto srá
***
Kdo: Posl. Patzel (německy)
Kde: 1920ns_ps_stenprot_290schuz_s290010.htm
Kdy: 1924-09-23
Co:  státní papír. To nemusí býti mnoho, ale je na každý způsob něco, i když jeho slosovatelnost byla rozvržena na 90 let, od 1. dubna 1935 až do konce roku 2024. Chci ovšem říci, že volba toho data přes rok 2000, o němž již jednou jeden americký spisovatel ve své knize "Pohled zpět z 
***
Kdo: Sen. Hlávka
Kde: 1925ns_se_stenprot_145schuz_s145010.htm
Kdy: 1928-10-11
Co:  nutným. Na př. já. Mně je 72

In [166]:
hledej(r'21\. století')

Kdo: Místopředseda vlády ČSSR a předseda Státní komise pro vědeckotechnický a investiční rozvoj J. Obzina
Kde: 1986fs_sn_stenprot_003schuz_s003001.htm
Kdy: 1987-05-19
Co: ěta. XVII. sjezdem Komunistické strany Československa vstoupila naše země do nového historického období - období zásadního zvratu v kvalitativním vývoji socialismu. Pojetí socialismu na přelomu 20. a 21\. století je definováno v dokumentech XXVII. sjezdu KSSS a XVII. sjezdu KSČ. V čem je historická specifičnost tohoto období ve srovnání se situací, která byla před 70 roky v sovětském Rusku nebo před 40 lety v
***
Kdo: Předsedkyně ČÚV SSM Zdeňka Tesařová
Kde: 1986cnr_stenprot_011schuz_s011017.htm
Kdy: 1988-06-28
Co: ků. Soudružky a soudruzi, mládež připravující se na své budoucí povolání tvoří více jak polovinu členské základny naší organizace. Její výchova a příprava na plnění úkolů společnosti na přelomu 20. a 21\. století klade vysoké nároky na jednotné působení školy, rodiny a všech výchovných činitelů. Rozvojem rů

In [184]:
hledej(r'[Rr]o[ck]\w{0,4} 2060')

Kdo: Poslanec Jan Schwippel
Kde: 2002ps_stenprot_044schuz_s044258.htm
Kdy: 2005-05-13
Co: dokumentováno, rozdíl mezi Evropou a Spojenými státy se nadále prohlubuje. Je to rozumné, protože podle analýz, které existují, tak některé parametry má Evropská unie šanci dohnat v roce 2030, jiné v [Rr]o[ck]\w{0,4} 2060 a jiné už vůbec ne. Teď bych se chtěl krátce zmínit o tom, proč si myslím, že dokument nebo strategie tohoto typu má velmi malé vyhlídky na to, být úspěšná. Za prvé jedná se o rámcový dokument. Bylo 
***
Kdo: Poslanec Břetislav Petr
Kde: 2006ps_stenprot_015schuz_s015184.htm
Kdy: 2007-06-13
Co: u nelogické. Převážná část nerostných surovin se dobývá v Čechách, a budeme-li posuzovat, jak dlouho hornictví v Čechách bude, tak jenom z hlediska hnědého uhlí, pokud nepřekročíme limity, to bude do [Rr]o[ck]\w{0,4} 2060, a pokud je snad překročíme, tak to bude až do roku 2100. A taky další část nerostných surovin, ať už jsou to slévárenské, sklářské písky, vápenec, bentonity, živce a dalš

In [182]:
hledej(r'[Rr]o[ck]\w{0,4} 2025')

Kdo: Poslanec Vladimír Špidla
Kde: 1996ps_stenprot_007schuz_s007040.htm
Kdy: 1996-12-18
Co: u uličkou, že musíme uvažovat o tom, že nezaměstnanost může dosáhnout vysloveně katastrofálních hodnot - hovoří se až o 80% nezaměstnanosti, jakkoliv je samozřejmě každá z těchto věcí velmi sporná do [Rr]o[ck]\w{0,4} 2025, že rozhodující pohled je přehodnotit názor na to, co je a co není práce. Vezmeme-li jenom trošku rozšířenou definici, která nebere práci jako něco, za co se dostává mzda, zjistíme, že zhruba 40 % hr
***
Kdo: Poslanec Rostislav Čevela
Kde: 1998ps_stenprot_032schuz_s032033.htm
Kdy: 2001-01-24
Co: mu odchodu do důchodu je prospěšné. Stárnutí populace je problém nejen naší země, ale celé Evropy - akcentovaný navíc nízkou porodností a předpokládaným nárůstem důchodců po roce 2005, nemluvě již po [Rr]o[ck]\w{0,4} 2025. A přátelé - zajímejme se o budoucnost, neboť v ní přece hodláme strávit zbytek svého života. Divize úřadu OSN pro populační trendy nedávno zveřejnila, že Evropané neu

In [192]:
hledej(r'[Rr]o[ck]\w{0,4} 2040')

Kdo: Posl. Leibl (německy)
Kde: 1925ns_ps_stenprot_030schuz_s030004.htm
Kdy: 1926-06-11
Co:  se dobytek zdražuje, těžce poškozuje. Za máslo, kterého docílím od krávy prostřední jakosti, pokud je po úhradě vlastní potřeby pro tři členy své rodiny mohu ještě prodati, vyzískal jsem v posledním roce 2040 korun. Kdyby tu bylo 5 nebo 6 hlav, nemohl bych vůbec prodávati máslo, takže také zde clo jen velmi málo nebo docela nic nenahradí. Ročně musím přikoupiti nejméně 8 až 9 měřic otrub jako hutnou píci.
***
Kdo: Místopředseda vlády a ministr práce a sociálních věcí ČR Vladimír Špidla
Kde: 1998ps_stenprot_039schuz_s039193.htm
Kdy: 2001-10-24
Co: ý počet nově narozených dětí znamená samozřejmě stárnutí populace. Znamená v dlouhé perspektivě ohrožení populace jako celku. Kdyby nedošlo ke změně a kdybychom byli ostrov - což jistě nejsme - kolem roku 2040 bychom překročili bod, který by znamenal v dalších několika desítkách let zánik českého národa jakožto biologické existence. Nemyslím si, že v d

In [194]:
hledej(r'[Rr]o[ck]\w{0,4} 2100')

Kdo: Poslankyně Anna Röschová
Kde: 1993ps_stenprot_009schuz_s009025.htm
Kdy: 1993-05-20
Co: Jako další vystoupil pan poslanec Matulka. Jeho návrh zní: V § 52 se navrhuje účinnost 1. července roku 2100. Nedoporučuji.
***
Kdo: Poslanec Miroslav Sládek
Kde: 1996ps_stenprot_012schuz_s012022.htm
Kdy: 1997-07-03
Co: , kdy se poslanci upisovali na odměnu pro dopadení pachatele tohoto činu. Vzhledem k tomu, že ten pachatel bude s ohledem na schopnosti Ministerstva vnitra jistě dopaden - nejspíše v roce 2050 nebo v roce 2100 - konec konců víte sami, že orgány činné v trestním řízení a v justici jsou velmi pomalé, když jde o vrahy, za čtyři roky nic nezvládnou a musí je pustit na svobodu, velmi rychlé v mém případě, to js
***
Kdo: Poslanec Stanislav Volák
Kde: 1998ps_stenprot_046schuz_s046141.htm
Kdy: 2002-02-01
Co: laci 45 %. Tak prosím, nelakujme to tady na růžovo. Ten problém je velmi vážný, ale důchodový problém není základním problémem. Základním problémem je demografie. Pokud to půjde takh

In [202]:
hledej(r'[Rr]o[ck]\w{0,4} 2025')

Kdo: Poslanec Vladimír Špidla
Kde: 1996ps_stenprot_007schuz_s007040.htm
Kdy: 1996-12-18
Co: u uličkou, že musíme uvažovat o tom, že nezaměstnanost může dosáhnout vysloveně katastrofálních hodnot - hovoří se až o 80% nezaměstnanosti, jakkoliv je samozřejmě každá z těchto věcí velmi sporná do roku 2025, že rozhodující pohled je přehodnotit názor na to, co je a co není práce. Vezmeme-li jenom trošku rozšířenou definici, která nebere práci jako něco, za co se dostává mzda, zjistíme, že zhruba 40 % hr
***
Kdo: Poslanec Rostislav Čevela
Kde: 1998ps_stenprot_032schuz_s032033.htm
Kdy: 2001-01-24
Co: mu odchodu do důchodu je prospěšné. Stárnutí populace je problém nejen naší země, ale celé Evropy - akcentovaný navíc nízkou porodností a předpokládaným nárůstem důchodců po roce 2005, nemluvě již po roce 2025. A přátelé - zajímejme se o budoucnost, neboť v ní přece hodláme strávit zbytek svého života. Divize úřadu OSN pro populační trendy nedávno zveřejnila, že Evropané neuživí důchodce již velmi 

In [212]:
hledej(r'[Rr]o[ck]\w{0,4} 2027')

Kdo: Poslankyně Hana Lagová
Kde: 1993ps_stenprot_032schuz_s032058.htm
Kdy: 1995-06-29
Co: hodnotu práce ženy v domácnosti, její péči o děti a zakomponovat ji do výdělkových zápočtových let. Druhý příklad ve vztahu k přechodnému období. V USA se zvyšuje věk ze 65 let na 67 let s horizontem roku 2027, ale s velmi důležitou poznámkou, že tato změna byla oznámena již v roce 1983. Několik málo slov k otázkám financování. Žádný z existujících systémů financování sociálního zabezpečení ve světě není e
***
Kdo: Poslanec Jan Bláha
Kde: 1996ps_stenprot_012schuz_s012020.htm
Kdy: 1997-07-03
Co:  se dobývací prostor Tušimice, o který se jedná, zvětšuje o 24, a další km2. Jedná se o hrubou neznalost tohoto dokumentu. Dále je tam ještě rozpor, kdy se hovoří o tom, že ukončení těžby se táhne do roku 2027, ale přesto v oficiálních údajích ministerstev se hovoří pro změnu s osmiletým rozdílem o oddálení až do roku 2035. Problém je, že žádné z ministerstev na mé otázky se nesnažilo důsledně prošetřit si


In [216]:
hledej(r'[Rr]o[ck]\w{0,4} 2030',n=5)

Kdo: Místopředseda vlády ČSSR a předseda Státní komise pro vědeckotechnický a investiční rozvoj J. Obzina
Kde: 1986fs_sn_stenprot_003schuz_s003002.htm
Kdy: 1987-05-19
Co: novitelné zdroje energie, paliv, surovin a materiálů, kromě hydroenergetiky, nedosáhnou v bilanci většího podílu než 1-2 % z celkové spotřeby prvotních zdrojů energie do roku 2000 a maximálně 10 % do roku 2030 až 2050. Současnou situaci v ČSSR charakterizují ve vztahu k těmto zdrojům tyto údaje: - malé vodní elektrárny šetří ročně 380 tis. tun měrného paliva, - geotermální energie 7 tis. tun měrného paliva
***
Kdo: Předseda FS A. Indra
Kde: 1986fs_slsn_stenprot_007schuz_s007005.htm
Kdy: 1987-12-15
Co: méně na úrovni dnešních tramvají, tzn. aby počet stížností a problémů dále a dále klesal. Mohl bych samozřejmě hovořit o životním prostředí. Když jsme sestavovali generel energetiky, ukázalo se, že v roce 2030 bude pravděpodobně v Praze ráj, protože se nebude uhlí, koks a nic podobného spalovat, a my tím směrem jdeme. A 

In [86]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 2020).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [84]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 2030).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [74]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 1997).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [16]:
df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 2000).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).filter(pl.col("roky") >= 50)

datum,roky
date,u32
1986-01-01,50
1987-01-01,54
1997-01-01,55
1998-01-01,74
1999-01-01,489
2000-01-01,468
2001-01-01,633
2002-01-01,300
2003-01-01,138


In [17]:
rozdil = df.drop_nulls(subset=["datum"]).explode("roky").sort(by="datum").with_columns((pl.col("roky") - pl.col("datum").dt.year()).alias("rozdil"))

In [18]:
rozdil_podil = rozdil.filter(pl.col("rozdil") < 0).group_by_dynamic(index_column='datum',every='1y').agg(pl.col("rozdil").len()).rename({'rozdil':'minulost'}).join(
    rozdil.filter(pl.col("rozdil") >= 5).group_by_dynamic(index_column='datum',every='1y').agg(pl.col("rozdil").len()).rename({'rozdil':'budoucnost'}), on='datum', how='left'
).with_columns((pl.col('budoucnost') / pl.col('minulost')).alias('podil'))

In [19]:
alt.Chart(
    rozdil_podil.to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

alt.Chart(...)

In [20]:
rozdil_podil.filter(pl.col('podil') > 0.15)

datum,minulost,budoucnost,podil
date,u32,u32,f64
1959-01-01,332,55,0.165663
1960-01-01,293,96,0.327645
1964-01-01,399,65,0.162907
1986-01-01,236,51,0.216102
1987-01-01,410,69,0.168293
2020-01-01,1884,313,0.166136


In [21]:
alt.Chart(
    rozdil.sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("rozdil").median()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("rozdil:Q")
)

alt.Chart(...)

In [22]:
rozdil.sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("rozdil").median()).sort(by="rozdil")

datum,rozdil
date,f64
1945-01-01,-7.0
1990-01-01,-6.0
1919-01-01,-4.0
1925-01-01,-3.0
1948-01-01,-3.0
2021-01-01,-3.0
2025-01-01,-3.0
1924-01-01,-2.0
1926-01-01,-2.0


In [23]:
alt.Chart(
    df.drop_nulls(subset=["datum"]).explode("roky").filter(pl.col("roky") == 2000).sort(by="datum").group_by_dynamic(index_column='datum',every='1y').agg(pl.col("roky").len()).to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X('datum:T'),alt.Y('roky:Q')
)

alt.Chart(...)

In [24]:
df.explode("roky").filter(pl.col("roky") == 2000).head().select(pl.col("text")).head(1).tail(1).item()

'Vážení páni kolegové! Mezi různými zákony, které mají zhojiti následky doby válečné a poválečné, je také zákon, jímž má býti zmocněna vláda, aby převzala pohledávky vzniklé z dodávek v době válečné. Nároky, o které zde jde, zakládají se na skutečných dodávkách a pracovních výkonech, které dosud zůstaly nezaplaceny, ačkoli republika Československá dostala do svého držení největší část dodaného materiálu. Uvádím jen mimochodem, že v zákoně a jeho důvodové zprávě nebylo nikde řečeno, že republika Československá se již dříve vyrovnala s částí věřitelů zvláště, jelikož při vyjednávání s vojenskou likvidační komisí ve Vídni, které se konalo za přítomnosti československého generálního konsulátu, značná část věřitelů, a to těch větších, byla pohnuta za jistého nátlaku k tomu, aby se zřekla poloviny svých pohledávek, aby se 50% prohlásilo za likvidní. To se týče hlavně oněch pohledávek, tak zvaných stornových, při nichž nedošlo více k odevzdání staré vojenské správě rakouské, uherské nebo rako

In [25]:
df.filter(pl.col('datum').dt.year() == 1986).explode("roky").filter(pl.col("roky") == 2000).head().select(pl.col("text")).head(1).tail(1).item()

'Vážený soudruhu generální tajemníku a prezidente republiky, vážený soudruhu předsedo Federálního shromáždění, vážené soudružky a soudruzi poslanci. Vláda Československé socialistické republiky předkládá Federálnímu shromáždění návrh státního závěrečného účtu československé federace za rok 1985 krátce po XVII. sjezdu KSČ, který podrobně zhodnotil dosažené výsledky v rozvoji ekonomiky a společnosti v uplynulém období a vytyčil program urychlení hospodářského a sociálního rozvoje na léta 1986-1990 a výhled do roku 2000. Rozhodujícím směrem zajištění této základní strategické linie je urychlení procesu intenzifikace na základě rozhodnějšího uplatňování nejnovějších výsledků vědeckotechnického rozvoje ve výrobní a společenské praxi, rozvoje socialistické ekonomické integrace, uskutečňování efektivních strukturálních přeměn ekonomiky a její modernizace. Ve své podstatě jde o zajištění nové kvality ekonomického růstu, což klade, jak bylo zdůrazněno na XVII. sjezdu strany, nové vyšší nároky n

In [26]:
alt.Chart(
    df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    pl.col("roky") - pl.col("datum").dt.year()
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('roky').max()
).to_pandas(),
    width=800
).mark_bar().encode(alt.X('datum:T'),alt.Y('roky:Q'))

alt.Chart(...)

In [27]:
df.explode("roky").drop_nulls(subset=['roky']).sort(by="roky",descending=True)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,roky
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,i32
"""Poslanec SN R. Chlad""","""1971fs_sn_stenprot_003schuz_s003009.htm""",null,"""Vážené soudružky poslankyně, vážení soudruzi poslanci, dovolte, abych jako člen přípravné pracovní skupiny přispěl k dnešnímu projednávání problematik…",5,"""fs""",1971,3,9,1973-10-30,"""FS ČSSR 1971-1976""","""3. schůze""",true,true,true,2400
"""Senátorka Alena Gajdůšková""","""2006se_stenprot_015schuz_s015008.htm""","""/sqw/detail.sqw?id=5703&sfo=6""","""Vážený pane předsedající, paní poslankyně, kolegy ě a kolegové, než se dostanu k tomu, co bych chtěla říci na téma probíraného návrhu, tak musím malin…",16,"""se""",2006,15,8,2008-07-16,"""PČR, Senát 2006-2008""","""15. schůze""",true,true,true,2400
"""Posl. Malý""","""1929ns_ps_stenprot_304schuz_s304003.htm""",null,"""Slavná sněmovno! Používám příležitosti projednávání státního rozpočtu republiky Československé k tomu, abych promluvil zde k největšímu státnímu podni…",3,"""ps""",1929,304,3,1933-12-02,"""NS RČS 1929-1935, Poslanecká sněmovna""","""304. schůze""",true,true,true,2253
"""Posl. dr. Hodža""","""1920ns_ps_stenprot_087schuz_s087008.htm""",null,"""Ctená snemovno! Keď v cele novej vlády máme príležitosť vítať šéfa našej zahraničnej politiky, a keď následkom toho určité hľadiská zahraničnej a svet…",4,"""ps""",1920,87,8,1921-10-19,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""87. schůze""",true,true,true,2250
"""Poslankyně Anna Čurdová""","""2002ps_stenprot_030schuz_s030149.htm""","""/sqw/detail.sqw?id=5263""","""Vážené kolegyně, kolegové, od pátku si připadám jak Alenka v říši divů a po dnešní diskusi chvilkami mám pocit, jako kdybych já - přesvědčený ateista …",3,"""ps""",2002,30,149,2004-03-30,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""30. schůze""",true,true,true,2250
"""Poslanec Pavel Hojda""","""2010ps_stenprot_003schuz_s003015.htm""","""/sqw/detail.sqw?id=366""","""Dobrý den, paní předsedkyně, vládo, milé dámy a vážení pánové. Dnes předstoupila vláda České republiky sestavená ze zástupců koaličních stran před Sně…",3,"""ps""",2010,3,15,2010-08-10,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""3. schůze""",true,true,true,2200
"""Ministr financí posl. Ďuriš""","""1954ns_stenprot_033schuz_s033009.htm""",null,"""Z nadplánovej hodinovej produktivity práce po odpočítaní spotrebovaného materiálu a odpise vo výške 2,17 Kčs zostáva na rozdelenie 0,47 Kčs. Na základ…",1,"""ns""",1954,33,9,1959-07-08,"""NS RČS 1954-1960""","""33. schůze""",true,true,false,2168
"""Ministr životního prostředí ČR Richard Brabec""","""2017ps_stenprot_040schuz_s040310.htm""","""https://www.vlada.cz/cz/clenove-vlady/richard-brabec-167027/""","""Děkuji za slovo. Vážený pane předsedající, kolegyně, kolegové, začnu tím, co řekl velmi správně pan kolega Adamec, a to je, že odpady byly, jsou a bud…",5,"""ps""",2017,40,310,2020-01-31,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true,true,2150
"""Místopředseda PSP Tomio Okamura""","""2017ps_stenprot_035schuz_s035412.htm""","""/sqw/detail.sqw?id=6105""","""Prosím vás, nepoučujte mě tady. Takže... (Připomínky z pléna.) Přesně, já jsem nezaslechl slovo protinávrh, paní poslankyně, takže buďme přesní. A nez…",13,"""ps""",2017,35,412,2019-11-07,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""35. schůze""",true,true,true,2119


In [28]:
df.explode("roky").drop_nulls(subset=['roky']).sort(by="roky",descending=True).head(1).select(pl.col("text")).item()

'Vážené soudružky poslankyně, vážení soudruzi poslanci, dovolte, abych jako člen přípravné pracovní skupiny přispěl k dnešnímu projednávání problematiky komplexní bytové výstavby několika poznatky z oblasti technologie a materiálně technického zásobování. Bytová výstavba je jedním z významných součinitelů stupně životní úrovně. Zvyšování životní úrovně je tedy ve svém důsledku závislé také na krytí stále rostoucích požadavků na kulturu bydlení. Vezmeme-li v úvahu nutnost náhrad za předpokládaný odpad bytového fondu, pak dojdeme k závěru, že i uspokojivý trend počtu předávaných bytů něco přes 100 000 ročně nebude asi stačit k plnému zabezpečování plánovaného růstu životní úrovně. Vždy při stávající produkci se předpokládá snížení nedostatku bytů ke konci roku 1975 na 293 tis., tj. toliko o 63 tis. bytů oproti roku 1970. Vedle pozitivních výsledků komplexní bytové výstavby, jak je zřejmo z dobře zpracované zprávy SPK o plnění plánu bytové výstavby a dalších úkolů, jsme svědky ještě celé 